# Optical character recognition (OCR) 
Project 4 Group 3


## Error Detection

In [1]:
!unzip data.zip

Archive:  data.zip
   creating: data/
   creating: data/ground_truth/
  inflating: data/ground_truth/group1_00000005.txt  
  inflating: data/ground_truth/group1_00000010.txt  
  inflating: data/ground_truth/group1_00000013.txt  
  inflating: data/ground_truth/group1_00000018.txt  
  inflating: data/ground_truth/group1_00000021.txt  
  inflating: data/ground_truth/group1_00000031.txt  
  inflating: data/ground_truth/group1_00000035.txt  
  inflating: data/ground_truth/group1_00000043.txt  
  inflating: data/ground_truth/group1_00000049.txt  
  inflating: data/ground_truth/group1_00000053.txt  
  inflating: data/ground_truth/group2_00000004.txt  
  inflating: data/ground_truth/group2_00000005.txt  
  inflating: data/ground_truth/group2_00000015.txt  
  inflating: data/ground_truth/group2_00000016.txt  
  inflating: data/ground_truth/group2_00000017.txt  
  inflating: data/ground_truth/group2_00000018.txt  
  inflating: data/ground_truth/group2_00000034.txt  
  inflating: data/ground_trut

In [0]:
import os
import glob
import re
from collections import Counter
import pandas as pd
import collections
import numpy as np
from sklearn.svm import SVC
import itertools
#import itertools
#nltk.download('punkt')

In [3]:
##### read ground_truth
ground_dir = glob.glob(os.path.join(os.getcwd(),'data/ground_truth','*.txt'))
ground_tokens = []
for gd in ground_dir:
    with open(gd) as ground_file:
        ground_raw = ground_file.read()
        ground_t = ground_raw.split()   
        ground_tokens += ground_t
print(len(ground_tokens))
print(ground_tokens[:20])

294841
['wa', 'reasonable,', 'although', 'it', 'substantially', 'increased', 'the', 'programs', 'and', 'authority', 'over', 'preset', 'law.', 'Following', 'is', 'a', 'brief', 'list', 'of', 'key']


In [0]:
tess_dir = glob.glob(os.path.join(os.getcwd(),'data/tesseract','*.txt'))
#ground_dir = glob.glob(os.path.join(os.getcwd(),'data/ground_truth','*.txt'))
file_name_gd = []
file_name_td = []
for gd, td in zip(ground_dir, tess_dir):
        with open(gd, encoding="utf8") as ground_file:    #, encoding="utf8"
            with open(td, encoding="utf8") as tess_file:                
                ground_r = list(ground_file.readlines()) 
                tess_r = list(tess_file.readlines())
                if len(tess_r) == len(ground_r):
                    file_name_td.append(td)
                    file_name_gd.append(gd)

In [0]:
# make sure all the lines have the same size.
ground_tokens=[]
tess_tokens=[]
for gd, td in zip(file_name_gd, file_name_td):
        with open(gd) as file1:
            with open(td) as file2:
                for line1,line2 in zip(file1,file2):
                    if len(line1)==len(line2):
                        for word1,word2 in zip(line1.split(),line2.split()):
                            ground_tokens.append(word1)
                            tess_tokens.append(word2)

In [6]:
print(tess_tokens[:30])
print(ground_tokens[:30])

['Ha', 'reas-', 'able,', 'although', '1:', 'substantlally', '1ncreased', 'the', 'programs', 'and', 'authorlty', 'preset', '1-H.', 'Followl', '.', '15', 'a', 'brlef', '115:', 'of', 'key', 'lssues', 'that', 'were', 'consldered', 'and', 'the', 'whlch', 'm-Ior', 'and']
['wa', 'reasonable,', 'although', 'it', 'substantially', 'increased', 'the', 'programs', 'and', 'authority', 'over', 'preset', 'law.', 'Following', 'is', 'a', 'brief', 'list', 'of', 'key', 'issues', 'that', 'were', 'considered', 'and', 'the', 'votes', 'which', 'major', 'and']


In [7]:
print(len(tess_tokens))
print(len(ground_tokens))

152143
152143


In [0]:
y = []
for gt, tt in zip(ground_tokens, tess_tokens):
        if gt == tt:
            y.append(0)   # 0 indicates correct
        else:
            y.append(1)   # 1 indicates error

In [0]:
X = pd.DataFrame(tess_tokens)   
from sklearn.model_selection import train_test_split  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)
ground_train = [ground_tokens[i] for i in X_train.index.tolist()]
ground_test = [ground_tokens[i] for i in X_test.index.tolist()]
X_train = X_train[0].tolist()
X_test = X_test[0].tolist()

In [11]:
print(X_train[:30])
print(ground_train[:30])
print(y_train[:30])

['type', '1n', '19557', 'Verltable', 'emlsslons', 'Hlth', 'Brussels', 'court,', 'are', 'organlzatlon.', 'for', 'truck', 'wlth', 'by', 'c', 'leglslatlon.', 'settlng', 'represent', 'on', 'and', 'program.', 'evaluatlng', 'at', 'necessary', 'on', 'on', 'the', '1nclude', 'Vice', 'be']
['of', 'in', '1965)', 'veritable', 'emissions', 'with', 'Brussels', 'court,', 'are', 'organization.', 'for', 'truck', 'with', 'the', 'f', 'legislation.', 'setting', 'represent', 'on', 'and', 'program.', 'evaluating', 'at', 'necessary', 'on', 'On', 'the', 'include', 'Vice', 'be']
[1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0]


In [0]:
# define character type
vowels = list('aeiou')
consonants = list("bcdfghjklmnpqrstvwxyz")
digits = list('0123456789')

def count_up(word):
    u = [x for x in word if x.isupper()]
    return len(u)

def count_low(word):
    l = [x for x in word if x.islower()]
    return len(l)

def count_cons_occur(word):  
    max_count=0
    count = 0
    curr = ''
    for c in word:
        if curr == c:
            count += 1
        else:
            max_count = max(max_count, count)
            curr = c
            count = 1
    max_count = max(max_count, count)
    return(max_count)

def count_cons_occur_consonants(word):  
    max_count=0
    count = 0
    curr = ''
    for c in word:
        if c in consonants:
            if curr == c:
                count += 1
            else:
                max_count = max(max_count, count)
                curr = c
                count = 1
    max_count = max(max_count, count)
    return(max_count)
    
def trim_word(word, from_start=1, from_end=1):
    return word[from_start:len(word) - from_end]

def unique(list1):  
    # intilize a null list 
    unique_list = []      
    # traverse for all elements 
    for x in list1: 
        # check if exists in unique_list or not 
        if x not in unique_list: 
            unique_list.append(x)
            
def safe_div(x,y):
    if y == 0:
        return 0
    return x / y

def get_bigram_freq(word, bi_dict):
    word = word.lower()
    bf = []
    for i in range(len(word)-1):
        key = word[i:i+2]
        bf.append(bi_dict[key])
    return(bf)

In [0]:
### find LB in ground_truth, prepared for bigram features
# ground_tokens of all words in lowercase
lower_ground_tokens = []
for tk in ground_tokens:
    tkl_g = tk.lower()
    lower_ground_tokens.append(tkl_g)

# A dict of all bigram frequencies
bigram_dict = collections.defaultdict(int)
for tk_g in lower_ground_tokens:
    for i in range(len(tk_g)-1):
        key = tk_g[i:i+2]
        bigram_dict[key] += 1

In [0]:
def feature_matrix(X):
    ##### Feature extracting
    # feature 1
    length = []

    # feature 2
    vowels_count = []
    consonants_count = []
    quotients_v_l = []
    quotients_c_l = []
    quotients_v_c = []

    # feature 3
    nonalpha = []

    # feature 4
    digits = []
    quotients_d_l = []

    # feature 5
    lowers = []
    uppers = []
    quotients_low_l = []
    quotients_up_l = []

    consecutive_occur = []
    quotients_cons_l = []
    quotients_la = [] # total number of vowels, consonants and digits
    consonants_secut = []
    fix_nonalpha = []

    # feature bigram
    bigr = []

    most_freq = []
    l2 = []
    
    for tk in X:
        ### feature 1
        l = len(tk)
        length.append(l)

        ### feature 2
        tkl = tk.lower()
        v_count = 0
        for v in tkl:
            if v in vowels:
                v_count +=1
        vowels_count.append(v_count)

        c_count = 0
        for c in tkl:
            if c in consonants:
                c_count +=1
        consonants_count.append(c_count)

        quotients_v_l.append(v_count/l)
        quotients_c_l.append(c_count/l)
        quot_v_c = safe_div(v_count,c_count)
        quotients_v_c.append(quot_v_c)

        ### feature 4
        d_count = len([d for d in tk if d in digits])
        digits.append(d_count)
        quotients_d_l.append(d_count/l)

        ### feature 3
        s_count = len([s for s in tk if s not in vowels or consonants or digits])
        nonalpha.append(s_count)

        ### feature 5
        low_count = count_low(tk)
        lowers.append(low_count)
        up_count = count_up(tk)
        uppers.append(up_count)
        quotients_low_l.append(low_count/l)
        quotients_up_l.append(up_count/l)

        ### feature 6
        cons_occur_count = int(count_cons_occur(tk))
        consecutive_occur.append(cons_occur_count)
        if cons_occur_count >= 3:
            quotients_cons_l.append(cons_occur_count/l)
        else:
            quotients_cons_l.append(0)

        ### feature 7
        la = v_count + c_count + d_count
        if s_count > la:
            quotients_la.append(1)
        else:
            quotients_la.append(0)

        ### feature 8
        consonance_cons_cccur_count = int(count_cons_occur_consonants(tk))
        if consonance_cons_cccur_count >= 6:
            consonants_secut.append(1)
        else:
            consonants_secut.append(0)

        ### feature 9
        tk_removed = trim_word(tk)
        k_count = len([k for k in tk_removed if k not in vowels or consonants or digits])
        if k_count >=3:
            fix_nonalpha.append(1)
        else:
            fix_nonalpha.append(0)  

        ### feature 10 bigram
        bf = get_bigram_freq(tk, bigram_dict)

        lower_tess_tokens = []
        tkl_t = tk.lower()
        lower_tess_tokens.append(tkl_t)

        n = len(set(lower_tess_tokens))
        big = (sum(bf)/10000)/n 
        bigr.append(big)

        ### feature 11 most frequent symbol
        i_count = Counter(tk).most_common(1)[0][1]
        if i_count >=3:
            most_freq.append(1)
        else:
            most_freq.append(0)

        ### feature 12 Non-alphabetical symbols: nonalpha/total symbols
        l1 = len([v for v in tk.lower() if v in vowels] + [c for c in tk.lower() if c in consonants])
        l2_count = l - l1
        quot_l2 = safe_div(l2_count,l1)
        l2.append(quot_l2)  
        
    ##### construct a feature dataframe for SVM
    df1 = pd.DataFrame({'length': length,
                        'vowels': vowels_count,
                        'consonants': consonants_count,
                        'quot v/l': quotients_v_l,
                        'quot c/l': quotients_c_l,
                        'quot v/c': quotients_v_c,
                        'nonalpha': nonalpha,
                        'digits': digits,
                        'quot d/l': quotients_d_l,
                        'lowers': lowers,
                        'uppers': uppers,
                        'quot low/l': quotients_low_l,
                        'quot up/l': quotients_up_l,
                        'cons occur': consecutive_occur,
                        'quot cons/l': quotients_cons_l,
                        'quot la': quotients_la,
                        'consonants_secut': consonants_secut,
                        'fix_nonalpha': fix_nonalpha,
                        'bigr': bigr,
                        'most_freq' : most_freq,
                        'l2': l2})
    return df1

In [0]:
X_feature_train = feature_matrix(X_train)
X_feature_test = feature_matrix(X_test)

In [16]:
svclassifier = SVC(kernel='rbf')  
svclassifier.fit(X_feature_train, y_train)  

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [0]:
import pickle
from google.colab import files

filename = 'svm_model.pkl'
pickle.dump(svclassifier, open(filename, 'wb'))
files.download('svm_model.pkl')
# Can load the model from output or upload to colab
# svm_model = pickle.load(open(filename, 'rb'))

In [0]:
#to predict
y_pred = svclassifier.predict(X_feature_test)  
# y_pred = svm_model.predict(X_feature_test)  

In [22]:
df_output = pd.DataFrame({'tokens_tesseract':X_test,
                          'Predict_by_SVM': y_pred})
print(df_output[:100])

    Predict_by_SVM tokens_tesseract
0                1               1n
1                0               on
2                0              and
3                1      Washlngton,
4                1                7
5                1               1:
6                0           Report
7                1               40
8                0          reached
9                0            could
10               1      Cozporatlon
11               0              the
12               0           recent
13               0             Vent
14               0          sesslon
15               0              are
16               0              and
17               0             able
18               0          between
19               1        commodlty
20               0              the
21               0              Mr.
22               1       Addltlonal
23               0               to
24               0         approach
25               0                a
26               1        Co

In [23]:
##### evaluation
#confustion Matrix
from sklearn.metrics import classification_report, confusion_matrix  
print(confusion_matrix(y_test,y_pred))  
print(classification_report(y_test,y_pred))  

[[16297  2093]
 [ 3459  8580]]
             precision    recall  f1-score   support

          0       0.82      0.89      0.85     18390
          1       0.80      0.71      0.76     12039

avg / total       0.82      0.82      0.82     30429



## Error correction

Given the detected word error, in order to find the best correction, we need to generating the candidate corrections: a dictionary or a database of legal n-grams to locate one or more potential correction terms. Then we need invoke some lexical-similarity measure between the misspelled string and the candidates or a probabilistic estimate of the likelihood of the correction to rank order the candidates. Here, we implement the positional binary digram method in the first reference paper. (https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=1672564}{positional binary digram)

In [0]:
import string

In [0]:
def keep_alphabet(tokens):
  # only retain alphabet
  out = []
  for l in tokens:
      l = l.lower()
      if l in set('abcdefghijklmnopqrstuvwxyz '):
          out.append(l)
  return ''.join(out)

In [0]:
# Replace a postion of a string
def replace_str_index(text,index=0,replacement=''):
    return '%s%s%s'%(text[:index],replacement,text[index+1:])

In [0]:
def correction(word, digrams):
  detect = 0
  beta = []
  for each in digrams:
    matrix = digrams[each]
    if matrix[string.ascii_lowercase.index(word[each[0]])][string.ascii_lowercase.index(word[each[1]])] == 0:
      detect += 1
      beta_each = set(each)
      if detect == 1:
        beta = beta_each
      else:
        beta = beta.intersection(beta_each)
  # print(beta)
  if len(beta) == 1:
    v_list = []
    for i in beta:
      position = i
    for j in range(len(word)):
      alpha_j = string.ascii_lowercase.index(word[j])
      if j < position:
        vector_j = digrams[(j, position)][alpha_j]
        v_list.append(vector_j)
      elif j > position:
        vector_j = [item[alpha_j] for item in digrams[(position, j)]]
        v_list.append(vector_j)
    v = v_list[0]
    for each in v_list:
      v = [a and b for a, b in zip(v, each)] 
    if sum(v) == 1:
      word = replace_str_index(word, position, string.ascii_lowercase[v.index(1)])
  # elif len(beta) == 2:
  return word

In [0]:
for i in range(len(ground_train)):
  ground_train[i] = keep_alphabet(ground_train[i])

In [0]:
# Create dictionary of digrams
token_by_len = collections.defaultdict(list)
digrams_by_len = collections.defaultdict(dict)
for w in ground_train:
  token_by_len[len(w)].append(w)
  
#print('Number of words of diffenrent length:')
#for key, value in token_by_len.items() :
#    print (key, len(value))

for length in token_by_len:
  for i, j in itertools.combinations(range(length), 2):
    key = (i, j)
    matrix = [[0] * 26 for _ in range(26)]
    for words in token_by_len[length]:
      matrix[string.ascii_lowercase.index(words[i])][string.ascii_lowercase.index(words[j])] = 1
    digrams_by_len[length][key] = matrix

In [90]:
# Number of 1 in digrams with given length
ae = 0
for each in digrams_by_len[10][(0,1)]:
  ae += sum(each)
print(ae)

144


In [0]:
# Clean the words
corrected_test = X_test.copy()
for i in range(len(corrected_test)):
  corrected_test[i] = keep_alphabet(corrected_test[i])
for i in range(len(ground_test)):
  ground_test[i] = keep_alphabet(ground_test[i])

In [129]:
# Make correction
for i in range(len(y_pred)):
  if y_pred[i] == 1:
    word_length = len(corrected_test[i])
    if word_length > 1:
      digrams_i = digrams_by_len[word_length]
      corrected_test[i] = correction(corrected_test[i], digrams_i)
      print(corrected_test[i])

washlngton
cozporatlon
commodlty
addltlonal
commlttee
envlronmental
mlnlmal
whlch
including
communlcatlons
begln
wllllngness
condltlons
publlc
alnlalz
publlsh
kentucky
th
steps
government
commlttee
operational
spokespersons
phthalate
nternatlonal
admlnlstratlve
approved
probably
lssues
understanding
whlch
contrlbui
categorlcal
ncluctnqr
projects
mportant
productlon
lssues
worklng
flrst
ways
regulatry
commitments
internatlonal
emlsslon
ndustry
rlsks
alternatives
compensation
companys
llkellhood
domestlc
chromium
hlth
subsector
yar
survlvors
analysls
commlttee
flcant
workshop
ndustry
polnt
forth
havlng
capltol
inltlated
commlttee
servlce
quallflcatlons
prlmary
pendlng
supports
barrlers
norganc
llablllty
examlning
hlth
alllerkslgnal
biotechnology
splnioff
mmendatlon
eulflll
nclude
hydroxytoluene
sult
and
dentcatlon
workshops
whlle
cozporatlons
coordlnators
conslstlng
hill
tanks
separately
presently
basls
news
technlcal
block
negotlatlons
unportant
formatlon
advertlslng
competitive
transpo

In [0]:
y_corrected = []
for gt, ct in zip(ground_test, corrected_test):
        if gt == ct:
            y_corrected.append(0)   # 0 indicates correct
        else:
            y_corrected.append(1)   # 1 indicates error

In [131]:
print(corrected_test[:30])
print(ground_test[:30])
print(X_test[:30])

['n', 'on', 'and', 'washlngton', '', '', 'report', '', 'reached', 'could', 'cozporatlon', 'the', 'recent', 'vent', 'sesslon', 'are', 'and', 'able', 'between', 'commodlty', 'the', 'mr', 'addltlonal', 'to', 'approach', 'a', 'commlttee', 'are', 'would', 'but']
['in', 'on', 'and', 'washington', '', 'it', 'report', '', 'reached', 'could', 'corporation', 'the', 'recent', 'vent', 'session', 'are', 'and', 'able', 'governments', 'commodity', 'the', 'mr', 'additional', 'to', 'approach', 'a', 'committee', 'are', 'would', 'but']
['1n', 'on', 'and', 'Washlngton,', '7', '1:', 'Report', '40', 'reached', 'could', 'Cozporatlon', 'the', 'recent', 'Vent', 'sesslon', 'are', 'and', 'able', 'between', 'commodlty', 'the', 'Mr.', 'Addltlonal', 'to', 'approach', 'a', 'Commlttee', 'are:', 'would', 'but']


In [132]:
print(sum(y_test))
print(sum(y_corrected))

12039
10462


## Performance measure

In [133]:
word_recall_tess = 1 - sum(y_test)/len(ground_test)
word_recall_corrected = 1 - sum(y_corrected)/len(ground_test)
# word_precision_tess = 1 - sum(y_test)/len(X_test)
# word_precision_corrected = 1 - sum(y_corrected)/len(X_test)
# char_recall_corrected = 

d = {'Tesseract': [word_recall_tess], 'Tesseract_with_postprocessing': [word_recall_corrected]}
OCR_performance_table = pd.DataFrame(data=d)
OCR_performance_table.rename(index={0: 'word_wise_recall'}, inplace=True)
print(OCR_performance_table)

                  Tesseract  Tesseract_with_postprocessing
word_wise_recall   0.604358                       0.656183
